In [15]:
from pytrends.request import TrendReq
import pandas as pd
from datetime import datetime, timedelta
import time
import os

# Connexion à Google Trends
pytrends = TrendReq(hl='fr-FR', tz=360)

# Paramètres
kw_list = ["Livraison", "Suivi colis", "Expédition colis", "Tarif colis", "Envoi colis", 
            "Livraison express", "Livraison domicile", "Délai livraison", "Retard livraison", 
            "Service de livraison", "Point relais", "colis"]
start_date = datetime(2017, 1, 1)
end_date = datetime(2023, 8, 1)
delta = timedelta(days=90)

# Créez un dossier pour stocker les fichiers mis en cache, s'il n'existe pas déjà
if not os.path.exists('cache'):
    os.makedirs('cache')

# Dictionnaire pour stocker les différents dataframes pour chaque terme
dfs_dict = {}

# Iterer sur chaque terme de la liste
for term in kw_list:
    all_dfs = []  # Liste pour stocker les dataframes pour ce terme
    current_start_date = start_date
    
    # Fractionnement de la plage de dates pour obtenir des données quotidiennes
    while current_start_date < end_date:
        next_end_date = current_start_date + delta
        if next_end_date > end_date:
            next_end_date = end_date

        cache_file = f"cache/{term}_{current_start_date.strftime('%Y%m%d')}_to_{next_end_date.strftime('%Y%m%d')}.csv"

        if os.path.exists(cache_file):
            df = pd.read_csv(cache_file, index_col=0)
        else:
            timeframe_str = current_start_date.strftime('%Y-%m-%d') + " " + next_end_date.strftime('%Y-%m-%d')
            pytrends.build_payload([term], cat=0, timeframe=timeframe_str, geo='FR', gprop='')
            df = pytrends.interest_over_time()
            
            # Cache the dataframe
            df.to_csv(cache_file)
            
            # Wait for 5 seconds
            time.sleep(5)

        all_dfs.append(df)
        current_start_date = next_end_date + timedelta(days=1)
    
    dfs_dict[term] = pd.concat(all_dfs, axis=0).drop(columns='isPartial', errors='ignore')

# Combine all individual dataframes into one dataframe
df_google_trends = pd.concat(dfs_dict.values(), axis=1)

print(df_google_trends)


            Livraison  Suivi colis  Expédition colis  Tarif colis  \
date                                                                
2017-01-01         76           13                 0           38   
2017-01-02         60           30                 0           64   
2017-01-03         56           31                 0           54   
2017-01-04         57           31                28           72   
2017-01-05         58           33                 0           46   
...               ...          ...               ...          ...   
2023-07-28         81           67                 0           52   
2023-07-29         80           54                 0           44   
2023-07-30         79           24                 0           67   
2023-07-31         93           65                 0           45   
2023-08-01         88           75               100           27   

            Envoi colis  Livraison express  Livraison domicile  \
date                                

In [16]:
df_google_trends.to_csv('data/google_trends.csv')